In [1]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [2]:
!pip install langchain

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install pypdf

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip3 install pinecone-client

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable


In [7]:
loader = PyPDFLoader("./aapl-10-k.pdf")

In [8]:
data = loader.load()

In [9]:
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[30].page_content)} characters in your document')

You have 78 document(s) in your data
There are 2639 characters in your document


In [10]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

C:\Users\spurt\AppData\Roaming\Python\Python311\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [11]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '546106c9-d7dd-439d-877b-157320aa3eda')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [12]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [13]:
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "langchaintest2"

In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [15]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [16]:
query = "What are the company's competition?"
docs = docsearch.similarity_search(query)

In [17]:
print(docs[0])

page_content='experienced little to no growth or contracted overall.Additionally, the Company faces signiﬁcant competition as competitors imitate the Company’s product features and applications withintheir products or collaborate to offer solutions that are more competitive than those they currently offer. The Company also expectscompetition to intensify as competitors imitate the Company’s approach to providing components seamlessly within their offerings orwork collaboratively to offer integrated solutions.The Company’s services also face substantial competition, including from companies that have signiﬁcant resources and experience andhave established service offerings with large customer bases. The Company competes with business models that provide content tousers for free. The Company also competes with illegitimate means to obtain third-party digital content and applications.The Company’s business, results of operations and ﬁnancial condition depend substantially on the Company’s

In [18]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [19]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [20]:
query = "What are the company's competition?"
docs = docsearch.similarity_search(query)

In [21]:
chain.run(input_documents=docs, question=query)

' The Company faces substantial competition in its markets from companies that have significant technical, marketing, distribution and other resources, as well as established hardware, software, and service offerings with large customer bases. The Company also competes with business models that provide content to users for free and use illegitimate means to obtain third-party digital content and applications.'

In [22]:
query = "What were the effects of the COVID-19 pandemic?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

" The effects of the COVID-19 pandemic included supply shortages that affected sales worldwide, disruption of the Company's component suppliers and manufacturing and logistical service providers, and a potential adverse impact on consumer demand for the Company's products and services."

In [23]:
query = "What are some of the business risks of the company?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

" Some of the business risks of the company include the success of new product and service introductions depending on a number of factors, the availability of application software for the company's products, the effective management of purchase commitments and inventory levels, the availability of products in appropriate quantities and at expected costs to meet anticipated demand, and the risk that new products and services may have quality or other defects or deficiencies. Additionally, the company's operations and performance depend significantly on global and regional economic conditions, and adverse macroeconomic conditions, including inflation, slower growth or recession, new or increased tariffs and other barriers to trade, changes to fiscal and monetary policy, and currency exchange rate fluctuations, can materially adversely affect the company's business, results of operations and financial condition."

In [24]:
query = "What is the human capital of the company?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The Company had approximately 164,000 full-time equivalent employees.'

In [25]:
query = "How is AAPL performing?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

" According to the graph, AAPL's cumulative total shareholder return for the five years ended September 24, 2022 was $411 when $100 was invested on September 29, 2017."

In [26]:
query = "How many people work at this company?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

" I don't know."

In [27]:
query = "How this company performing?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Apple Inc. has experienced little to no growth or contracted overall, as evidenced by the cumulative total shareholder return graph which shows that $100 invested on September 29, 2017 was worth $411 on September 24, 2022.'

## Test on 10-K document questions

In [28]:
query = "How does the company generate its revenue? What are its main products or services?"
docs = docsearch.similarity_search(query)
model_answer = chain.run(input_documents=docs, question=query)

' The company generates its revenue through direct and indirect distribution channels. Its main products and services include hardware, operating systems, software applications, and related services.'

In [30]:
human_answer ='The Company designs, manufactures and markets smartphones (iPhones), personal computers (Macbooks and Macs), tablets (iPads), wearables and accessories (Airpods, Apple TV, Apple Watch), and sells a variety of related services.'



In [31]:
query = 'Does the company operate in multiple geographic regions or industries?'
docs = docsearch.similarity_search(query)
model_answer = chain.run(input_documents=docs, question=query)
model_answer

' The company operates in multiple geographic regions, including the Americas, Europe, Greater China, Japan, and Rest of Asia Pacific.'

In [32]:
human_answer = 'Yes, the Company has international operations with sales outside the U.S.  In addition, the Company’s global supply chain is large and complex and a majority of the Company’s supplier facilities, including manufacturing and assembly sites, are located outside the U.S.'

In [36]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Convert the texts into TF-IDF vectors
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([model_answer, human_answer])

# Calculate the cosine similarity between the vectors
similarity = cosine_similarity(vectors)
print(similarity[0][1])

0.3401864999931753


In [42]:
#test_answer = ' The company operates in several geographic regions including the Americas, Europe, Greater China, Japan, and Rest of Asia Pacific.'

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([test_answer, model_answer])

# Calculate the cosine similarity between the vectors
similarity = cosine_similarity(vectors)
print(similarity)

[[1.         0.91011139]
 [0.91011139 1.        ]]


In [43]:
def compare_strings(text1, text2):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([text1, text2])
    # Calculate the cosine similarity between the vectors
    similarity = cosine_similarity(vectors)
    print(similarity)

In [44]:
query = "Are there any significant changes or developments in the company's business model, strategy, or competitive landscape?"
docs = docsearch.similarity_search(query)
model_answer = chain.run(input_documents=docs, question=query)
human_answer = "During the COVID-19 pandemic, the Company’s component suppliers and manufacturing and logistical service providers have experienced disruptions, resulting in supply shortages that affected sales worldwide"
compare_strings(model_answer, human_answer)

[[1.         0.15438321]
 [0.15438321 1.        ]]


In [45]:
model_answer

" No, the company's business model, strategy, and competitive landscape remain largely unchanged."